In [1]:
!pip install apache-beam[interactive]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install apache_beam[dataframe]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import apache_beam as beam

p1 = beam.Pipeline()
visit_count = (
 
  p1
  |beam.dataframe.io.read_csv('/content/ball_by_ball_data_commentary.csv')

  |beam.Map(lambda record: record.split(','))
  |beam.Filter(lambda record: record[3] == 'cardio')
  |beam.Map(lambda record: (record[1], 1))
  |beam.CombinePerKey(sum)
 
  |beam.io.WriteToText('out_data.txt')
)

p1.run()

In [8]:
ilines = (
  p1
  |beam.Create([1,2,3])
  |beam.io.WriteToText('outfile.txt')
)

In [9]:
beam.Create([1, 2, 3]) | beam.Map(lambda x: [x, 'macroMeta']) 

# Three lists output: [[1, 'macroMeta'], [2, 'macroMeta'], [3, 'macroMeta']]


beam.Create([1, 2, 3]) | beam.FlatMap(lambda x: [x, 'macroMeta']) 

#Six single elements output: [1, 'macroMeta', 2, 'macroMeta', 3, 'macroMeta']

<_ChainedPTransform(PTransform) label=[Create|FlatMap(<lambda at 33773647.py:6>)] at 0x7fdcc2559990>

In [13]:


import apache_beam as beam


p = beam.Pipeline()


input_collection = (

    p
    | "Read input data" >> beam.io.ReadFromText('/content/ball_by_ball.txt')
    | "Split rows into columns" >> beam.Map(lambda record: record.split(','))
                  )

AFG_count = (
    input_collection
    | 'Filter AFG team' >> beam.Filter(lambda record: record[2] == 'AFG')
    | 'Pair each AFG with 1' >> beam.Map(lambda record: ("AFG, " +record[1], 1))
    | 'Aggregate AFG' >> beam.CombinePerKey(sum)
    )

SL_count = (
              input_collection
              | 'Get all SL team' >> beam.Filter(lambda record: record[3] == 'SL')
              | 'Pair each SL team with 1' >> beam.Map(lambda record: ("SL, " +record[1], 1))
              | 'Aggregate SL' >> beam.CombinePerKey(sum)
          )

output =(
        (AFG_count,SL_count)
  | beam.Flatten()
  | beam.io.WriteToText('data/both')
)

In [16]:
import apache_beam as beam

class CustomTransform(beam.PTransform):
  def expand(self, input_coll):
 
    a = (
      input_coll
                      | 'Group and sum' >> beam.CombinePerKey(sum)
                      | 'filter regular patients' >> beam.Filter(filter_on_count)
                      | 'Regular patient' >> beam.Map(format_output)
           
  )
    return a

def SplitRow(element):
  return element.split(',')

def filter_on_count(element):
 name, count = element
 if count > 1:
  return element
 def format_output(element):
  name, count = element
 #return ', '.join((name.encode('ascii'),str(count),'Regular employee'))
#return ', '.join((name,str(count),'Regular Patient'))

p = beam.Pipeline()

input_collection = (

              p
                    | "Read from text file" >> beam.dataframe.io.read_csv('/content/ball_by_ball_data_commentary.csv')
                    | "Split rows" >> beam.Map(SplitRow)
                  )

AFG_count = (
                    input_collection
                    | 'Get all AFG team' >> beam.Filter(lambda record: record[2] == 'AFG')
                    | 'Pair each AFG with 1' >> beam.Map(lambda record: ("AFG, " +record[1], 1))
                    | 'composite AFG team' >> CustomTransform()
                    | 'Write results for AFG' >> beam.io.WriteToText('AFG_output.txt')
                )

SL_count = (
              input_collection
              | 'Get all HR dept persons' >> beam.Filter(lambda record: record[2] == 'SL')
              | 'Pair each SL team with 1' >> beam.Map(lambda record: ("SL, " +record[1], 1))
              | 'composite SL team' >> CustomTransform()
              | 'Write results for SL' >> beam.io.WriteToText('SL_output.txt')
          )
p.run()